#### Description of the code 
This code identifies the critical event in SHRP2, which is related to the limits of lateral acceleration. 

The description of the event is as follows:
the $\textbf{acceleration}$ is greater than $0.75g$ for two consecutive time steps. Multiple trigers in a 20 seconds interval is considered one event.

In [ ]:
# import datetime
def identify_critical_events(data):
    critical_acceleration = 7.5 # 7.5 m/s²
    critical_duration = 2  # number of time steps for 0.2 seconds at 10 Hz
    join_window = 10  # number of time steps for 1 seconds at 10 Hz
    frequency = 10  # 10 Hz

    critical_events = []
    event_start = None
    event_end = None
    event_duration = 0

    for i in range(len(data)):
        # Check if lateral acceleration exceeds the critical acceleration
        if abs(data[i]['lateral_acceleration']) >= critical_acceleration:
            event_duration = event_duration + 1
            if event_duration >= critical_duration:
                event_start = data[i - event_duration + 1]['time']
                event_end = data[i]['time']
                print('start of the event', event_start)
        else:
            if event_start is not None:
                if critical_events != []:
                    if event_start - critical_events[-1][1]  <= join_window/10:
                        print('merge')
                        critical_events[-1] = (critical_events[-1][0], event_end)
                        event_start = None
                        event_end = None
                        event_duration = 0
                    else:
                        print('new event')
                        critical_events.append((event_start, event_end))
                        event_start = None
                        event_end   = None
                        event_duration = 0
                else:
                    print('new event')
                    critical_events.append((event_start, event_end))
                    event_start = None
                    event_end   = None
                    event_duration  = 0

    if event_start is not None: # if the last event is not closed
        critical_events.append((event_start, len(data)/frequency ))

    return critical_events, event_start, event_end

# Example data
start_time = 0.0
time_interval = 0.10
time_stamp = 0

example_data = []
time = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
        0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 
        1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 
        2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 
        2.6, 2.7, 2.8, 2.9, 3.0]
lateral_accelerations = [0.5, 7.7, 8.0, 8.1, 0.6, 0.5, 7.5, 7.6, 0.4, 0.3, 
                         0.2, 0.8, 0.2, 0.3, 0.1, 0.4, 0.3, 0.2, 0.3, 0.4, 
                         0.5, 0.7, 0.6, 0.5, 0.4, 0.3, 9.2, 9.1, 9.0, 0.3]

example_data = [{'time': time[i], 'lateral_acceleration': lateral_accelerations[i]} for i in range(len(time))]

### Identifying critical events
events, start, end = identify_critical_events(example_data)
print("Critical events (start_index, end_index):", events)

# Optional: Convert indices to time labels
# event_times = [(example_data[start]['time'], example_data[end]['time']) for start, end in events]
# print("Critical event times:", event_times)


#### Test on inD dataset

import pandas as pd 


In [1]:
import pandas as pd

In [2]:
# Function for inD dataset
def identify_critical_events(data, critical_acc = 0.75):
    critical_acceleration = critical_acc # 7.5 m/s²
    critical_duration = 2  # number of time steps for 0.2 seconds at 10 Hz
    join_window = 10  # number of time steps for 1 seconds at 10 Hz
    frequency = 10  # 10 Hz

    critical_events = []
    event_start = None
    event_end = None
    event_duration = 0

    for i in range(len(data)):
        # Check if lateral acceleration exceeds the critical acceleration
        if abs(data.iloc[i]['latAcceleration']) >= critical_acceleration:
            event_duration = event_duration + 1
            if event_duration >= critical_duration:
                event_start = data.iloc[i - event_duration + 1]['trackLifetime']
                event_end = data.iloc[i]['trackLifetime']
                # print('start of the event', event_start) # For checking if events are detected correctly
        else:
            if event_start is not None:
                if critical_events != []:
                    if event_start - critical_events[-1][1]  <= join_window/10:
                        # print('merge')
                        critical_events[-1] = (critical_events[-1][0], event_end)
                        event_start = None
                        event_end = None
                        event_duration = 0
                    else:
                        # print('new event')
                        critical_events.append((event_start, event_end))
                        event_start = None
                        event_end   = None
                        event_duration = 0
                else:
                    # print('new event')
                    critical_events.append((event_start, event_end))
                    event_start = None
                    event_end   = None
                    event_duration  = 0

    if event_start is not None: # if the last event is not closed
        critical_events.append((event_start, len(data)/frequency ))

    return critical_events, event_start, event_end



In [3]:
path = 'inD-dataset-v1.0/data/22_tracks.csv'
data = pd.read_csv(path)

The original code iterates on one vehicle. Here, we need to also iterate on different vehicles (track IDs)

In [4]:
acc_data = data[['trackId', 'trackLifetime', 'latAcceleration']]

In [5]:
data_grouped = acc_data.groupby('trackId')
for id, group in data_grouped:
    events, start, end = identify_critical_events(group, critical_acc = 0.1)
    print('trackId', id)
    print("Critical events (start_index, end_index):", events)
    if id == 10: break

trackId 0
Critical events (start_index, end_index): [(0.0, 78.0), (136.0, 175.0)]
trackId 1
Critical events (start_index, end_index): [(0.0, 80.0), (108.0, 129.0), (153.0, 264.0), (296.0, 320.0), (341.0, 356.0), (377.0, 390.0), (470.0, 488.0), (518.0, 538.0), (581.0, 60.0)]
trackId 2
Critical events (start_index, end_index): [(0.0, 141.0), (187.0, 230.0)]
trackId 3
Critical events (start_index, end_index): [(0.0, 29.0), (89.0, 117.0), (166.0, 202.0), (237.0, 270.0), (280.0, 297.0), (305.0, 324.0), (389.0, 406.0), (420.0, 446.0), (501.0, 543.0), (558.0, 635.0), (650.0, 666.0), (691.0, 713.0), (723.0, 744.0), (754.0, 773.0), (787.0, 81.1)]
trackId 4
Critical events (start_index, end_index): [(19.0, 80.0)]
trackId 5
Critical events (start_index, end_index): [(0.0, 23.0), (42.0, 65.0), (74.0, 106.0), (585.0, 605.0), (614.0, 636.0), (649.0, 677.0), (697.0, 719.0), (741.0, 757.0), (841.0, 908.0), (934.0, 950.0), (972.0, 991.0), (1057.0, 1104.0), (1127.0, 1159.0), (1182.0, 1186.0), (1313.0, 1